In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
import time
from model import *
from data import *

![U-NET](https://lmb.informatik.uni-freiburg.de/Publications/2015/RFB15a/u-net-architecture.png)


U-Net，基於Fully Convolutional Networks(全捲積網路)，輸入和輸出都是圖，沒有全連接層。
https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/

## Train your Unet with membrane data
membrane data is in folder Membrane/, it is a binary classification task.

The input shape of image and mask are the same :(batch_size,rows,cols,channel = 1)

### Train with data generator

In [7]:
# early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# history = model.fit(x_train, y_train, validation_split=0.2, epochs=1000, batch_size=20, callbacks=[early_stop

In [2]:
tic = time.time()
BatchSize = 1

data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')

myGene = trainGenerator(BatchSize,'Membrane','image','label',data_gen_args,save_to_dir = None)
#myGene = trainGenerator(BatchSize,'Fixed','image','label',data_gen_args,save_to_dir = None)
#myGene = trainGenerator(BatchSize,'Mitochondrion','image','label',data_gen_args,save_to_dir = None)
model = unet()
# model = simple_unet_model()
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('UNet_Membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
#model_checkpoint = ModelCheckpoint('UNet_Fixed.hdf5', monitor='loss',verbose=1, save_best_only=True)
#model_checkpoint = ModelCheckpoint('UNet_Mitochondrion.hdf5', monitor='loss',verbose=1, save_best_only=True)
#model.fit_generator(myGene,steps_per_epoch=2000,epochs=10,callbacks=[model_checkpoint])
model.fit(myGene,steps_per_epoch=1000,epochs=5,callbacks=[model_checkpoint])
toc = time.time()
print("Time=" + str((toc-tic)) + "sec")

Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.
Epoch 1/5



KeyboardInterrupt



### Train with npy file

In [ ]:
#imgs_train,imgs_mask_train = geneTrainNpy("data/membrane/train/aug/","data/membrane/train/aug/")
#model.fit(imgs_train, imgs_mask_train, batch_size=2, nb_epoch=10, verbose=1,validation_split=0.2, shuffle=True, callbacks=[model_checkpoint])

### test your model and save predicted results

In [ ]:
testGene = testGenerator("Membrane/test")
model = unet()
model.load_weights("UNet_Membrane.hdf5")
#results = model.predict_generator(testGene,30,verbose=1)
results = model.predict(testGene,30,verbose=1)
saveResult("Membrane/test",results)